# Day 7: Amplification Circuit
https://adventofcode.com/2019/day/7

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools

Instruction set and intcode interpreter V5, based on version V4 from day 11. 

The reason I do this is because when I first came to part 2 I skipped it because I couldn't figure out how to do the feedback loop. Shame, I know. So I went on solving the next days problems. Until I got to day 11, when inspiration came to me. So I made the IntcodeInterpreter interact with primitive devices. In this case I will use "devices" to connect one amplifier with the next.

The former implementation I did to solve part 1 used the very program as temporal data storage. Something I could not make peace with... so I came back and rewrote all day 7 with interpreter V4. Later on I made version V5 with step-by-step executions. Shouldn't be a problem as intcode machines are retrocompatible.

In [ ]:
class IntcodeParameter():
    POSITION = 0
    IMMEDIATE = 1
    RELATIVE = 2
    
    value = None
    mode = None
    
    def __init__(self, value, mode):
        self.value = value
        self.mode = mode
        assert mode in [IntcodeParameter.POSITION, IntcodeParameter.IMMEDIATE, IntcodeParameter.RELATIVE], \
            f'UNKNOWN PARAMETER MODE {mode}'

    def checkBoundedParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            assert self.value >= 0 and self.value < len(program.code), \
                f'position parameter out of bounds. Position: {self.value}, Program size {len(program.code)}'
        elif self.mode == IntcodeParameter.RELATIVE:
            realPos = self.value + program.relative_base
            assert realPos >= 0 and realPos < len(program.code), \
                'relative parameter out of bounds. Position: {self.value}, Relative base {program.relative_base}, Program size {len(program.code)}'
        return True
        
    def evaluateParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return program.code[self.value]
        elif self.mode == IntcodeParameter.IMMEDIATE:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return program.code[self.value + program.relative_base]
        else:
            raise Exception('UNKNOWN PARAMETER MODE {}. Value {}. Program {}'.format(self.mode, self.value, program))
    
    def getParameterPosition(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return self.value + program.relative_base
        elif self.mode == IntcodeParameter.IMMEDIATE:
            raise Exception(f'Immediate parameter mode {self.value} cannot be used as position!')
        else:
            raise Exception(f'UNKNOWN PARAMETER MODE {self.mode}. Value {self.value}. Program {program}')
    
    def toString(self, program):
        return '[value {}, mode {}, evaluation {}]'.format(self.value, self.mode, self.evaluateParameter(program))

In [ ]:
class IntcodeInstruction():
    endComputations = False
    parameters = []
    
    def mustEndComputations(self):
        return self.endComputations
    
    def getInstructionSize(self):
        return len(self.parameters) + 1 #parameters + opcode
    
    def getNextInstructionPointer(self, program):
        newIP = program.IP + self.getInstructionSize()
        assert newIP >= 0 and newIP < len(program.code), \
            f'new instruction pointer out of bounds. New IP {newIP}, last IP {program.IP}, Program size {len(program.code)}'
        return newIP
    
    def extractParameters(self, program):
        pass
    
    def performComputation(self, program):
        pass
    
    def checkParameters(self, program):
        for param in self.parameters:
            param.checkBoundedParameter(program)
    
    def __loadParameters(self, program):
        self.parameters = self.extractParameters(program)
        self.checkParameters(program)
    
    def compute(self, program):
        self.__loadParameters(program)
        self.performComputation(program)
    
    def toString(self, program):
        msg = f'{self.__class__.__name__} '
        for param in self.parameters:
            msg += param.toString(program)
        
        return (msg)

class OneParameterInst(IntcodeInstruction):
    def extractParameters(self, program):
        parameters = []
        modes = program.code[program.IP] // 100
        parameters.append(IntcodeParameter(program.code[program.IP + 1], modes % 10))
        
        return parameters

class TwoParametersInst(OneParameterInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #1 parameters + 1 result
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 2], modes % 10))
        
        return parameters
    
class ThreeParameterInst(TwoParametersInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #2 parameters + 1 result
        modes //= 10
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 3], modes % 10))
        
        return parameters
    
class AdditionInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) + self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class MultiplyInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) * self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue

class LessThanInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) < self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class EqualsInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) == self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
        
class InputInst(OneParameterInst):
    def performComputation(self, program):
        n = input('Solicitando dato de entrada:')
        
        # TODO: Validar dato!!! Debe ser entero, dado que luego podrá ser
        # usado para operar con él o ser un opcode, etc..
        
        program.code[self.parameters[0].getParameterPosition(program)] = int(n)

class OutputInst(OneParameterInst):
    def performComputation(self, program):
        print('>>>OUTPUT: {}'.format(self.parameters[0].evaluateParameter(program)))

class JumpIfTrueInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfTrueInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) != 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class JumpIfFalseInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfFalseInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) == 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class AdjustRelativeBaseInst(OneParameterInst):
    def performComputation(self, program):
        shift = self.parameters[0].evaluateParameter(program)
        program.relative_base += shift
    
class QuitInst(IntcodeInstruction):
    def __init__(self):
        self.endComputations = True

    def extractParameters(self, program):
        return []
        
    def performComputation(self, program):
        pass

In [ ]:
class IntcodeProgram():
    MAX_SIZE_IMAGE = 2000
    IP = None
    code = None
    relative_base = None
    
    def __init__(self, code, relative_base = 0):
        self.IP = 0
        self.relative_base = relative_base
        self.code = self.__expand_code(code) #maybe code.copy() ?

    def __expand_code(self, code):
        assert len(code) < IntcodeProgram.MAX_SIZE_IMAGE, \
            f'Program is too big! program size {len(code)}. Max size {IntcodeProgram.MAX_SIZE_IMAGE}'
        return code + ( (IntcodeProgram.MAX_SIZE_IMAGE - len(code)) * [0]  )

In [ ]:
class IntcodeInterpreterV4():
    instructions = None
    program = None
    
    def __init__(self):
        self.instructions = {}
        # TODO: Inject instruction set
        # Arithmetic
        self.instructions[1]  = AdditionInst()
        self.instructions[2]  = MultiplyInst()
        
        # IO
        self.instructions[3]  = InputInst()
        self.instructions[4]  = OutputInst()
        
        #Logic
        self.instructions[7]  = LessThanInst()
        self.instructions[8]  = EqualsInst()
        
        #Jump
        self.instructions[5]  = JumpIfTrueInst()
        self.instructions[6]  = JumpIfFalseInst()

        #Mem
        self.instructions[9]  = AdjustRelativeBaseInst()
        
        #STOP
        self.instructions[99] = QuitInst()

    def __getNextInstructionType(self, program):
        opcode = program.code[program.IP]
        return opcode % 100
        
    def getInstruction(self, program):
        opcode = self.__getNextInstructionType(program)
        if opcode in self.instructions:
            instruction = self.instructions[opcode]
            return instruction
        else:
            raise Exception('UNKNOWN opcode {} ({}) at position {}.\nPROGRAM: {}'.format(opcode, program.code[program.IP], program.IP, program.code))
        return None

    def __loadProgram(self, ext_program):
        self.program = IntcodeProgram(ext_program)
    
    def compute(self, ext_program):
        self.__loadProgram(ext_program)
    
        while True:
            isLastStep = self.nextStep()
            if isLastStep:
                break
            
           
        return self.program.code
    
    def beginStep(self, ext_program):
        self.__loadProgram(ext_program)
    
    def nextStep(self):
        assert self.program is not None, 'There is no program loaded!'
        # Extract next instruction            
        instruction = self.getInstruction(self.program)

        if instruction.mustEndComputations():
            return True
        else:
            # Otherwise, compute instruction
            instruction.compute(self.program)
            # Change IP (Instruction Pointer)
            self.program.IP = instruction.getNextInstructionPointer(self.program)
            
        return False
    
    def getNextInstructionType(self):
        assert self.program is not None, 'There is no program loaded!'
        
        return self.__getNextInstructionType(self.program)
        

We'll create new input and output instructions that will read data from and write data to a primitive device (a queue) that will act as a pipe. So an amplifier writes is output signal on its output pipe and the next amplifier will read data from that pipe.

Each machine need two data to work, phase and input signal. The input signal will be written by the previos amplifier but we need the phase to be written as well. So, the input pipe will be shared between the IO IntcodeInstructions and the amplifiers so each amplifier can write its phase into the pipe.

The first amplifier will have an unconnected pipe to read from, and will write the two values into it so its machine can access them.

The last amplifier will have an unconnected pipe to write to, so the amplifier can access its output signal, that will be the final output signal of the amplification process.


### Devices

In [ ]:
class Device():
    def read(self):
        raise Exception('Read operation not defined for this device!!')
        
    def write(self, data):
        raise Exception('Write operation not defined for this device!!')

class PipeDevice(Device):
    buffer = None
    label = None
    
    def __init__(self, label):
        self.buffer = []
        self.label = label
    
    def read(self):
        assert self.buffer is not None, f"Device {self.label}: Buffer is not ready! Can't read from it!"
        assert len(self.buffer) > 0, f"Device {self.label}: Buffer is empty. Can't read any data from it!"
        
        data = self.buffer[0]
        self.buffer = self.buffer[1:]
        
#         print(f'\tDevice {self.label}: READ {data}', 'remaining buffer is:', self.buffer)
        
        return data
    
    def write(self, data):
        assert self.buffer is not None, f"Device {self.label}: Buffer is not ready! Can't write on it!"
        
        self.buffer.append(data)
        
#         print(f'\tDevice {self.label}: WRITE {data}', 'buffer is:', self.buffer)

### New IO instructions

In [ ]:
class IOInputInst(OneParameterInst):
    device = None
    
    def setDevice(self, device):
        self.device = device
    
    def performComputation(self, program):
        assert self.device is not None, "Can't read from device: No device present!"
        
        n = self.device.read()
        
        # TODO: Validate data!!! It must be an int because it can be used later as an operator, an opcode...
        program.code[self.parameters[0].getParameterPosition(program)] = int(n)

class IOOutputInst(OneParameterInst):
    device = None
    
    def setDevice(self, device):
        self.device = device
        
    def performComputation(self, program):
        assert self.device is not None, "Can't write to device: No device present!"
        
        data = self.parameters[0].evaluateParameter(program)
        
        self.device.write(data)

In [ ]:
class Amplifier():
    amplifierCode = None
    machine = None
    program = None
    
    inputPipe = None
    outputPipe = None
    
    phaseReady = False
    
    def __init__(self, code, machine, program):
        self.amplifierCode = code
        self.machine = machine
        self.program = program

    def setInputChannel(self, inputInstr, pipeDevice):
        self.machine.instructions[3] = inputInstr()
        self.machine.instructions[3].setDevice(pipeDevice)
        self.inputPipe = pipeDevice
    
    def setOutputChannel(self, outputInstr, pipeDevice):
        self.machine.instructions[4] = outputInstr()
        self.machine.instructions[4].setDevice(pipeDevice)
        self.outputPipe = pipeDevice
    
    def setPhase(self, phase):
        assert self.inputPipe is not None, \
            f"Amplifier {self.amplifierCode}, Can't set phase, input device is not ready"
        self.inputPipe.write(phase)
        self.phaseReady = True
    
    def setInputSignal(self, inputSignal):
        assert self.inputPipe is not None, \
            f"Amplifier {self.amplifierCode}, Can't set input signal, input device is not ready"
        self.inputPipe.write(inputSignal)
        
    def getOutputSignal(self):
        assert self.outputPipe is not None, \
            f"Amplifier {self.amplifierCode}, Can't get output signal, output devide is not ready"
        return self.outputPipe.read()
    
    def amplifySignal(self):
        assert self.phaseReady, f"Amplifier {self.amplifierCode}. Phase is not ready!!"
        
        self.machine.compute(self.program)

Wiring the amplifiers like this:

```
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
```

In [ ]:
def getAmplifiersPart1(program):
    amplifiers = []
    
    ampA = Amplifier('A', IntcodeInterpreterV4(), program.copy())
    pipeOA = PipeDevice('OA')
    ampA.setInputChannel(IOInputInst, pipeOA)
    pipeAB = PipeDevice('AB')
    ampA.setOutputChannel(IOOutputInst, pipeAB)
    amplifiers.append(ampA)
    
    ampB = Amplifier('B', IntcodeInterpreterV4(), program.copy())
    ampB.setInputChannel(IOInputInst, pipeAB)
    pipeBC = PipeDevice('BC')
    ampB.setOutputChannel(IOOutputInst, pipeBC)
    amplifiers.append(ampB)
    
    ampC = Amplifier('C', IntcodeInterpreterV4(), program.copy())
    ampC.setInputChannel(IOInputInst, pipeBC)
    pipeCD = PipeDevice('CD')
    ampC.setOutputChannel(IOOutputInst, pipeCD)
    amplifiers.append(ampC)

    ampD = Amplifier('D', IntcodeInterpreterV4(), program.copy())
    ampD.setInputChannel(IOInputInst, pipeCD)
    pipeDE = PipeDevice('DE')
    ampD.setOutputChannel(IOOutputInst, pipeDE)
    amplifiers.append(ampD)

    ampE = Amplifier('E', IntcodeInterpreterV4(), program.copy())
    ampE.setInputChannel(IOInputInst, pipeDE)
    pipeEO = PipeDevice('EO')
    ampE.setOutputChannel(IOOutputInst, pipeEO)
    amplifiers.append(ampE)
   
    return amplifiers

In [ ]:
def performPermutation(amplifiers, permutation):
    ampA = None
    ampE = None
    
    for ampID in range(len(amplifiers)):
        amp = amplifiers[ampID]
        phase = permutation[ampID]
        amp.setPhase(phase)
        
        if amp.amplifierCode == 'A':
            ampA = amp
        elif amp.amplifierCode == 'E':
            ampE = amp

    ampA.setInputSignal(0)
            
    for amp in amplifiers:
        amp.amplifySignal()
    
    output_signal = ampE.getOutputSignal()
    
    return output_signal

In [ ]:
def getMaxSignal(program, ampGenerator, permutationPerformer, phaseValues):
    amplifiers = ampGenerator(program)
    
    # calculate permutations
    permutations = list(itertools.permutations(phaseValues))
    
    bestPermutation = None
    maxSignal = None
    for permutation in permutations:
        output_signal = permutationPerformer(amplifiers, permutation)
        print('Permutation {} yields {}'.format(permutation, output_signal))
        
        if output_signal == None:
            continue
        
        if bestPermutation == None:
            bestPermutation = permutation
            maxSignal = output_signal
            print(f'New maxSignal {output_signal} using permutation {permutation}')
        else:
            if output_signal > maxSignal:
                maxSignal = output_signal
                bestPermutation = permutation
                print(f'New maxSignal {output_signal} using permutation {permutation}')

    return bestPermutation, maxSignal

### Tests

#### Test 1
 * 3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
 * Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0)

In [ ]:
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

amplifiers = getAmplifiersPart1(program.copy())


print('Testing exact permutation')
permutation = [4, 3, 2, 1, 0]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy(), getAmplifiersPart1, performPermutation,[0, 1, 2, 3, 4])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

#### Test 2
* 3,23,3,24,1002,24,10,24,1002,23,-1,23, 101,5,23,23,1,24,23,23,4,23,99,0,0
* Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4)

In [ ]:
program = [3,23,3,24,1002,24,10,24,1002,23,-1,23, 101,5,23,23,1,24,23,23,4,23,99,0,0]

amplifiers = getAmplifiersPart1(program)

print('Testing exact permutation')
permutation = [0, 1, 2, 3, 4]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy(), getAmplifiersPart1, performPermutation,[0, 1, 2, 3, 4])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

#### Test 3
* 3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
* Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2)

In [ ]:
program = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]

amplifiers = getAmplifiersPart1(program)

print('Testing exact permutation')
permutation = [1,0,4,3,2]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy(), getAmplifiersPart1, performPermutation,[0, 1, 2, 3, 4])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

### Solution

In [ ]:
input_7 = r'data\aoc2019-input-day7.txt'
with open(input_7, 'r') as f:
    data7 = [int(data) for data in f.read().split(',') if len(data) > 0]

In [ ]:
print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(data7.copy(), getAmplifiersPart1, performPermutation,[0, 1, 2, 3, 4])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

>>>OUTPUT: 17406

## Part 2

Wiring the amplifiers like this:

```
     O-------O  O-------O  O-------O  O-------O  O-------O
0 -+->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
```

We'll extend our amplifiers to change the way they amplify the signal. Looking into the examples provided, we see that it first read the phase, then it enters a loop. Inside the loop, it reads a value again (input signal in this case) computes something, writes the value in its output pipe and loop again. But it can only perform one iteration because its buffer is empty the second time. It needs amplifier E to write into its input pipe. So, instead of running the full program with the compute method, we'll go step by step. We will make sure that every time we run into an IOInputInst it has enough data into its buffer to work. If it does not, then execution is "paused" and the next amplier comes into play. 

We will loop through every amplifier until their program ends (opcode 99). In order to do so, we will store the amplifiers inside a queue. We will dequeue the amplifiers and perform computations until it can't go on because of lack of data, or the program ends. Then, if the last executed instruction wasn't a 99, it gets queued again. This goes on until the queue is empty (all amplifiers ended their code). When this happens, the output of amplifier E is the final output signal.

In [ ]:
class FeedBackAmplifier(Amplifier):
    def __init__(self, code, machine, program):
        super().__init__(code, machine, program)

    def amplifySignal(self):
        assert self.phaseReady, f"Amplifier {self.amplifierCode}. Phase is not ready!!"
        
        if self.machine.program is None:
            self.machine.beginStep(self.program)
        
        while True:
            nexttype = self.machine.getNextInstructionType()
            if nexttype == 99:
                return True
            elif nexttype != 3:
                self.machine.nextStep()
            else:
                if len(self.inputPipe.buffer) > 0:
                    self.machine.nextStep()
                else:
                    #Not enough data to read
                    return False
    
    def resetAmplifier(self):
        self.machine.program = None

Next is the wiring for part 2. Notice a new pipe device that connects amplifiers A and E.

In [ ]:
def getAmplifiersPart2(program):
    amplifiers = []
    
    ampA = FeedBackAmplifier('A', IntcodeInterpreterV4(), program.copy())
#     pipeOA = PipeDevice('OA')
#     ampA.setInputChannel(IOInputInst, pipeOA)
    pipeAB = PipeDevice('AB')
    ampA.setOutputChannel(IOOutputInst, pipeAB)
    amplifiers.append(ampA)
    
    ampB = FeedBackAmplifier('B', IntcodeInterpreterV4(), program.copy())
    ampB.setInputChannel(IOInputInst, pipeAB)
    pipeBC = PipeDevice('BC')
    ampB.setOutputChannel(IOOutputInst, pipeBC)
    amplifiers.append(ampB)
    
    ampC = FeedBackAmplifier('C', IntcodeInterpreterV4(), program.copy())
    ampC.setInputChannel(IOInputInst, pipeBC)
    pipeCD = PipeDevice('CD')
    ampC.setOutputChannel(IOOutputInst, pipeCD)
    amplifiers.append(ampC)

    ampD = FeedBackAmplifier('D', IntcodeInterpreterV4(), program.copy())
    ampD.setInputChannel(IOInputInst, pipeCD)
    pipeDE = PipeDevice('DE')
    ampD.setOutputChannel(IOOutputInst, pipeDE)
    amplifiers.append(ampD)

    ampE = FeedBackAmplifier('E', IntcodeInterpreterV4(), program.copy())
    ampE.setInputChannel(IOInputInst, pipeDE)
    pipeEA = PipeDevice('EA')
    ampE.setOutputChannel(IOOutputInst, pipeEA)
    amplifiers.append(ampE)
    
    #CLosing the loop!
    ampA.setInputChannel(IOInputInst, pipeEA)
   
    return amplifiers

Evaluating a permutation is modified to allow the feedback loop.

In [ ]:
def performPermutationPart2(amplifiers, permutation):
    ampA = None
    ampE = None
    
    for ampID in range(len(amplifiers)):
        amp = amplifiers[ampID]
        phase = permutation[ampID]
        amp.setPhase(phase)
        
        if amp.amplifierCode == 'A':
            ampA = amp
        elif amp.amplifierCode == 'E':
            ampE = amp

    ampA.setInputSignal(0)

    #Feedback loop!!!
    feedBackLoop = list(amplifiers)
    while len(feedBackLoop) > 0:
        amp = feedBackLoop[0]
        feedBackLoop = feedBackLoop[1:]
#         print(f'FL: Amplifier {amp.amplifierCode}')                
        
        finalizedAmplification = amp.amplifySignal()
        if not finalizedAmplification:
            feedBackLoop.append(amp)
    
    output_signal = ampE.getOutputSignal()
    
    for amp in amplifiers:
        amp.resetAmplifier()
    
    return output_signal

### Test

#### Test 1
 * 3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26, 27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5
 * Max thruster signal 139629729 (from phase setting sequence 9,8,7,6,5)

In [ ]:
program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26, 27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

amplifiers = getAmplifiersPart2(program)

print('Testing exact permutation')
permutation = [9,8,7,6,5]
result = performPermutationPart2(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy(), getAmplifiersPart2, performPermutationPart2, [5, 6, 7, 8, 9])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

#### Test 2
 * 3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54, -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4, 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10
 * Max thruster signal 18216 (from phase setting sequence 9, 7, 8, 5, 6)

In [ ]:
program = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54, -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4, 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]

amplifiers = getAmplifiersPart2(program)

print('Testing exact permutation')
permutation = [9,7,8,5,6]
result = performPermutationPart2(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy(), getAmplifiersPart2, performPermutationPart2, [5, 6, 7, 8, 9])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

### Solution

In [ ]:
print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(data7.copy(), getAmplifiersPart2, performPermutationPart2, [5, 6, 7, 8, 9])
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

>>>SOLUTION: 1047153